# CRISP-DM Pipeline for HousingData.xlsx

This notebook implements the CRISP-DM process on the provided `HousingData.xlsx` dataset.

Phases we will cover:
- Business Understanding: define objectives and success metrics (predict housing value or key target).
- Data Understanding: load data, inspect schema, quality, distributions, correlations.
- Data Preparation: handle missing values, encode categoricals, scale numerics, feature/target selection, split.
- Modeling: train baseline and tree-based models with cross-validation.
- Evaluation: compare models using MAE, RMSE, R², and diagnostics.
- Deployment: persist cleaned data and model artifacts, record insights and next steps.

Notes:
- The notebook attempts to infer a reasonable target column (e.g., `SalePrice`, `Price`, `MEDV`). You can override it by setting `TARGET_COLUMN` below.
- Paths are absolute to avoid working directory issues.


In [ ]:
# Setup: imports and configuration
import os
import sys
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Modeling
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# Paths
DATA_PATH = "/Users/prisciladenise/Documents/YEAR 3 /Big data/Assignment 2/NO1/HousingData.xlsx"
ARTIFACT_DIR = "/Users/prisciladenise/Documents/YEAR 3 /Big data/Assignment 2/NO1/artifacts"
os.makedirs(ARTIFACT_DIR, exist_ok=True)

# Default target guess; can be changed after initial inspection
TARGET_COLUMN = None  # e.g., "SalePrice", "Price", "MEDV"
SEED = 42
np.random.seed(SEED)



In [ ]:
# Load the Excel data and basic inspection
xl = pd.ExcelFile(DATA_PATH)
print("Sheets:", xl.sheet_names)

# Try to auto-select the first sheet
sheet = xl.sheet_names[0]
df_raw = xl.parse(sheet)
print("Shape:", df_raw.shape)
df_raw.head(3)


In [ ]:
# Schema, missingness, and basic stats
print("Columns (n=", len(df_raw.columns), "):\n", list(df_raw.columns))
print("\nDtypes:\n", df_raw.dtypes)

missing = df_raw.isna().mean().sort_values(ascending=False)
print("\nMissingness (fraction):\n", missing.head(20))

display(df_raw.describe(include=[np.number]).T.head(20))
display(df_raw.describe(include=[object]).T.head(20))


In [ ]:
# Attempt to infer target column
possible_targets = [
    "SalePrice", "Price", "MEDV", "median_house_value", "price",
    "Sale_Price", "Target", "y"
]

if TARGET_COLUMN is None:
    for col in df_raw.columns:
        if col in possible_targets:
            TARGET_COLUMN = col
            break

print("TARGET_COLUMN:", TARGET_COLUMN)

# If still None, pick last numeric column as a fallback (can be changed later)
if TARGET_COLUMN is None:
    numeric_cols = df_raw.select_dtypes(include=[np.number]).columns.tolist()
    TARGET_COLUMN = numeric_cols[-1] if numeric_cols else None
    print("Fallback TARGET_COLUMN:", TARGET_COLUMN)

assert TARGET_COLUMN is not None, "Could not infer target column. Please set TARGET_COLUMN manually."



In [ ]:
# Correlations and distributions for numeric features
numeric_df = df_raw.select_dtypes(include=[np.number])

plt.figure(figsize=(10, 6))
sns.histplot(numeric_df[TARGET_COLUMN].dropna(), kde=True)
plt.title(f"Distribution of {TARGET_COLUMN}")
plt.show()

corr = numeric_df.corr(numeric_only=True)
plt.figure(figsize=(10, 8))
sns.heatmap(corr, cmap="coolwarm", center=0)
plt.title("Correlation heatmap (numeric)")
plt.show()

# Top correlations with target
if TARGET_COLUMN in corr.columns:
    target_corr = corr[TARGET_COLUMN].drop(TARGET_COLUMN).sort_values(ascending=False)
    display(target_corr.head(10))
    display(target_corr.tail(10))


In [ ]:
# Data preparation: features/target split, preprocessing pipelines, train/test split

# Separate features and target
X = df_raw.drop(columns=[TARGET_COLUMN])
y = df_raw[TARGET_COLUMN]

# Identify column types
numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = X.select_dtypes(exclude=[np.number]).columns.tolist()

# Preprocessing for numeric data: impute median, then scale
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

# Preprocessing for categorical data: impute most_frequent, then one-hot encode
categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features)
    ]
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=SEED
)

print({
    "num_features": len(numeric_features),
    "cat_features": len(categorical_features),
    "X_train": X_train.shape,
    "X_test": X_test.shape
})


In [ ]:
# Modeling: define models and cross-validation

models = {
    "LinearRegression": LinearRegression(),
    "Ridge": Ridge(alpha=1.0, random_state=SEED),
    "RandomForest": RandomForestRegressor(n_estimators=300, random_state=SEED),
    "GradientBoosting": GradientBoostingRegressor(random_state=SEED)
}

cv = KFold(n_splits=5, shuffle=True, random_state=SEED)

results = []
for name, model in models.items():
    pipe = Pipeline(steps=[("preprocess", preprocessor), ("model", model)])
    neg_mae = cross_val_score(pipe, X_train, y_train, scoring="neg_mean_absolute_error", cv=cv)
    neg_rmse = cross_val_score(pipe, X_train, y_train, scoring="neg_root_mean_squared_error", cv=cv)
    r2 = cross_val_score(pipe, X_train, y_train, scoring="r2", cv=cv)
    results.append({
        "model": name,
        "mae_mean": -neg_mae.mean(),
        "mae_std": neg_mae.std(),
        "rmse_mean": -neg_rmse.mean(),
        "rmse_std": neg_rmse.std(),
        "r2_mean": r2.mean(),
        "r2_std": r2.std()
    })

results_df = pd.DataFrame(results).sort_values(by=["rmse_mean"])  # lower is better
results_df


In [ ]:
# Fit best model on full training set and evaluate on test set
best_row = results_df.iloc[0]
best_model_name = best_row["model"]
print("Best by CV RMSE:", best_model_name)

best_model = models[best_model_name]
final_pipe = Pipeline(steps=[("preprocess", preprocessor), ("model", best_model)])
final_pipe.fit(X_train, y_train)

preds = final_pipe.predict(X_test)
mae = mean_absolute_error(y_test, preds)
rmse = mean_squared_error(y_test, preds, squared=False)
r2 = r2_score(y_test, preds)

print({"test_mae": mae, "test_rmse": rmse, "test_r2": r2})

plt.figure(figsize=(6,6))
sns.scatterplot(x=y_test, y=preds)
plt.xlabel("Actual")
plt.ylabel("Predicted")
plt.title(f"Actual vs Predicted ({best_model_name})")
lims = [min(y_test.min(), preds.min()), max(y_test.max(), preds.max())]
plt.plot(lims, lims, 'r--')
plt.show()


In [ ]:
# Feature importance (tree-based models only)
if best_model_name in ["RandomForest", "GradientBoosting"]:
    # Need fitted model; extract feature names after preprocessing
    # Fit once more on all data to get consistent encoder columns
    final_pipe.fit(X, y)
    # Get feature names from ColumnTransformer
    ohe = final_pipe.named_steps["preprocess"].named_transformers_["cat"].named_steps["onehot"] if len(categorical_features) else None
    cat_feature_names = (ohe.get_feature_names_out(categorical_features).tolist() if ohe is not None else [])
    feature_names = numeric_features + cat_feature_names

    model = final_pipe.named_steps["model"]
    importances = getattr(model, "feature_importances_", None)
    if importances is not None:
        imp_df = pd.DataFrame({"feature": feature_names, "importance": importances}) \
                 .sort_values("importance", ascending=False).head(20)
        plt.figure(figsize=(10,6))
        sns.barplot(data=imp_df, x="importance", y="feature")
        plt.title("Top feature importances")
        plt.show()
    else:
        print("Model has no feature_importances_.")
else:
    print("Feature importance skipped: best model is not tree-based.")


In [ ]:
# Persist cleaned data and model artifacts

# Save processed train/test splits (numerical arrays are fine for downstream)
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

np.save(os.path.join(ARTIFACT_DIR, "X_train.npy"), X_train_processed)
np.save(os.path.join(ARTIFACT_DIR, "X_test.npy"), X_test_processed)
np.save(os.path.join(ARTIFACT_DIR, "y_train.npy"), y_train.to_numpy())
np.save(os.path.join(ARTIFACT_DIR, "y_test.npy"), y_test.to_numpy())

# Save the final pipeline using joblib
import joblib
joblib.dump(final_pipe, os.path.join(ARTIFACT_DIR, "final_model.joblib"))

print("Artifacts saved to:", ARTIFACT_DIR)


## Findings, Insights, and Next Steps

- Business Objective: predict housing value (or chosen target) from available features.
- Data Quality: reviewed missingness, types, distributions; applied imputations and encodings.
- Modeling: compared linear and tree-based models with 5-fold CV; selected best by RMSE.
- Evaluation: reported MAE, RMSE, R² on holdout; visualized actual vs predicted and importances (if applicable).

Next Steps:
- Validate assumptions (linearity, leakage). Engineer domain features (e.g., age, ratios, interactions).
- Hyperparameter tuning (GridSearchCV/Optuna) for the best model.
- Calibrate error with business impact; define acceptance criteria.
- Package the pipeline for batch or API inference; monitor drift.

To change target: set `TARGET_COLUMN` earlier and re-run.
